In [3]:
import pandas as pd
import numpy as np
import math
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler

In [4]:

def clean_data(df):
    df.columns=[e.lower().replace(' ', '_') for e in df.columns]
    df=df.drop(['loan_date_end'], axis=1)
    df['a/w'] = df['a/w'].fillna('unknown_a/w')
    df['d/w'] = df['d/w'].fillna('unknown_d/w')
    df = df[df['joined'].isna()==False]  
    df['composure'] = df['composure'].fillna(df['composure'].mean())
    to_inches = lambda x: int(x.split("'")[0]) * 12 + int(x.split("'")[1].strip('"'))
    df['height'] = df['height'].apply(to_inches)
    df['weight'] = pd.to_numeric(df['weight'].str.replace('lbs', ''))
    df['joined'] = pd.to_datetime(df['joined'],errors='coerce')
    df['joined'] = df['joined'].dt.year
    def convert_currency(value):
        if 'K' in value:
            return float(value.replace('€', '').replace('K', '')) * 1000
        elif 'M' in value:
            return float(value.replace('€', '').replace('M', '')) * 1000000
        else:
            return float(value.replace('€', ''))
    df['value'] = df['value'].apply(convert_currency)
    def convert_currency(wage):
        if 'K' in wage:
            return float(wage.replace('€', '').replace('K', '')) * 1000
        else:
            return float(wage.replace('€', ''))
    df['wage'] = df['wage'].apply(convert_currency)
    def convert_currency(hits):
        if 'K' in hits:
            return float(hits.replace('K', '')) * 1000
        else:
            return float()
    df['hits'] = df['hits'].apply(convert_currency)
    def convert_currency(release_clause):
        if 'K' in release_clause:
            return float(release_clause.replace('€', '').replace('K', '')) * 1000
        elif 'M' in release_clause:
            return float(release_clause.replace('€', '').replace('M', '')) * 1000000
        else:
            return float(release_clause.replace('€', ''))
    df['release_clause'] = df['release_clause'].apply(convert_currency)
    df['ir']=df['ir'].str.replace('★','')
    df['w/f']=df['w/f'].str.replace('★','')
    df['sm']=df['sm'].str.replace('★','')
    df['ir'] = pd.to_numeric(df['ir'])
    df['w/f'] = pd.to_numeric(df['w/f'])
    df['sm'] = pd.to_numeric(df['sm'])
    df_to_clean=['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']
    for col in df_to_clean:
        df[col] = df[col].str.split('+').str[0]
        df[col] = df[col].str.split('-').str[0]
    for col in df_to_clean:
        df[col] = pd.to_numeric(df[col])
    X_cat1=df[['bp', 'foot', 'a/w', 'd/w']]
    pd.get_dummies(X_cat1, drop_first=True).head()
    from sklearn.preprocessing import OneHotEncoder
    pd.DataFrame(OneHotEncoder(drop='first').fit_transform(X_cat1).toarray(),
             columns=OneHotEncoder(drop='first').fit(X_cat1).get_feature_names(input_features=X_cat1.columns)).head()
    X_cat1=pd.get_dummies(X_cat1, drop_first=True)
    num=df._get_numeric_data()
    df=pd.concat([num, X_cat1], axis=1)
    return df


In [5]:
df = pd.read_csv('./fifa21_validate.csv')

In [6]:
df = clean_data(df)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
df

,id,age,height,weight,growth,joined,value,wage,release_clause,attacking,...,bp_RW,bp_RWB,bp_ST,foot_Right,a/w_Low,a/w_Medium,a/w_unknown_a/w,d/w_Low,d/w_Medium,d/w_unknown_d/w
0,219461,23,74,194,7,2018,975000.0,5000.0,0.0,230,...,0,0,0,1,1,0,0,0,0,0
1,221896,22,68,154,5,2020,1200000.0,3000.0,2200000.0,298,...,0,0,0,1,0,0,0,1,0,0
2,247428,19,74,176,17,2018,120000.0,500.0,249000.0,48,...,0,0,0,1,0,1,0,0,1,0
3,255120,16,75,170,23,2020,160000.0,500.0,464000.0,215,...,0,0,0,1,0,1,0,0,1,0
4,215556,24,74,170,5,2019,2300000.0,13000.0,4300000.0,295,...,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,21,68,143,11,2020,325000.0,1000.0,731000.0,197,...,0,0,0,0,0,0,0,0,1,0
1995,241223,21,77,176,9,2017,190000.0,700.0,285000.0,77,...,0,0,0,1,0,1,0,0,1,0
1996,210930,27,67,146,0,2019,8000000.0,9000.0,12000000.0,332,...,0,0,0,0,0,0,0,0,1,0
1997,162993,34,71,176,0,2020,140000.0,4000.0,245000.0,268,...,0,0,0,1,0,1,0,0,1,0


### Normalizing

In [8]:
import pickle
MinMaxtransformer=pickle.load(open('MinMaxtransformer.p', 'rb'))

In [9]:
MinMaxtransformer

MinMaxScaler()

In [10]:
lm=pickle.load(open('model.p', 'rb'))
lm

LinearRegression()

In [11]:
y=df['ova']
X=df.drop(['ova'], axis=1)

In [12]:
X_normalized = MinMaxtransformer.transform(X)
print(X_normalized.shape)
X_normalized = pd.DataFrame(X_normalized,columns=X.columns)
X_normalized.head()

(1996, 110)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- foot_Right
Feature names seen at fit time, yet now missing:
- foot

  warnings.warn(message, FutureWarning)


,id,age,height,weight,growth,joined,value,wage,release_clause,attacking,...,bp_RW,bp_RWB,bp_ST,foot_Right,a/w_Low,a/w_Medium,a/w_unknown_a/w,d/w_Low,d/w_Medium,d/w_unknown_d/w
0,0.847033,0.225806,0.666667,0.694444,7.0,80.72,34750.285714,0.000064,0.000,1.442911e-06,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.856432,0.193548,0.266667,0.324074,5.0,80.80,42786.000000,0.000038,8.800,1.869511e-06,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.954982,0.096774,0.666667,0.527778,17.0,80.72,4214.571429,0.000006,0.996,3.011292e-07,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.984672,0.000000,0.733333,0.472222,23.0,80.80,5643.142857,0.000006,1.856,1.348808e-06,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.831960,0.258065,0.666667,0.472222,5.0,80.76,82071.714286,0.000167,17.200,1.850690e-06,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


### Train Testing

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1596, 110)
(400, 110)
(1596,)
(400,)


In [14]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [15]:
from sklearn.metrics import r2_score

predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.9254870627518754

In [16]:
predictions_test = lm.predict(X_test)
r2_score(y_test, predictions_test)

0.9049365656288344

In [17]:
y_test[:5]

1011    56
1723    72
591     76
1350    70
595     66
Name: ova, dtype: int64

In [18]:
predictions_test[:5]

array([54.78794265, 69.64310529, 79.60732189, 70.69754172, 62.96105874])

In [19]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test,predictions_test)
mse

4.294513819429536

In [20]:
rmse = np.sqrt(mean_squared_error(y_test,predictions_test))
rmse

2.0723208775258564

In [21]:
y_test.mean()

66.9525

In [22]:
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, predictions_test)
print(mae)

1.5823666370255425


In [23]:
rmse = math.sqrt(mse)
print(rmse)

2.0723208775258564
